In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.patches import Patch
from matplotlib.offsetbox import AnchoredText
import pandas as pd
import numpy as np
from Color import *
from operator import truediv, sub
from matplotlib.lines import Line2D
from matplotlib.ticker import FormatStrFormatter

# Matplotlib font configuration
from MatplotlibFonts import *

# Helpers
from BrokenPlot import *

# Change the font size
rcParams.update({"font.size": 13})

In [3]:
%reload_ext autoreload
from BenchmarkConfiguration import *
from Color import *

In [4]:
RESULT_DIR='../benchmarks/logs'

In [5]:
Benchmarks = [
    'coremark',
    'picojpeg',
    'aes',
    'crc',
    'dijkstra',
    'sha',
    'adpcm',
    'quicksort',
    'towers'
]

Configurations = [
    'clank',
    'prowl',
    'nacho_pw_stcont'
]


OnTimes = [
    '0',
    '250000',    # 5ms @50MHz
    '500000',   # 10ms @ 50MHz
    '2500000',  # 50ms @ 50MHz
    '5000000',  # 100ms @ 50MHz
#    '25000000', # 500ms @ 50MHz
]

cache_size = '512-2'

In [6]:
# Load all the results into a dictionary

def pfStr(ontime):
    return str(int(int(ontime)/2)) + '-' + ontime

def loadResult(Results, benchmark, config, cache, pf):
    if cache == '0-0':
        filename = RESULT_DIR + '/' + benchmark + '-uninstrumented-run-' + config + '-' + pf + '-O3-final'
    else:
        filename = RESULT_DIR + '/' + benchmark + '-uninstrumented-run-' + config + '-' + cache + '-' + pf + '-O3-final'
    
    #print(filename)
    
    if pf not in Results:
        Results[pf] = {}
    if cache not in Results[pf]:
        Results[pf][cache] = {}
    if config not in Results[pf][cache]:
        Results[pf][cache][config] = {}
    
    # Add the results
    Results[pf][cache][config][benchmark] = dict(pd.read_csv(filename, delimiter=":", header = None).values)
    
Results = {}

for benchmark in Benchmarks:
    for ontime in OnTimes:
        pf_str = pfStr(ontime)
        loadResult(Results, benchmark, 'nacho_pw_stcont', cache_size, pf_str)
        loadResult(Results, benchmark, 'prowl', cache_size, pf_str)
        loadResult(Results, benchmark, 'clank', '0-0', pf_str)
    


In [7]:
# Create a dataframe
PlotDataDF = {}

for ontime in OnTimes:
    pf_str = pfStr(ontime)

    PlotDataDF[ontime] = {}
    PlotDataDF[ontime]['nacho_pw_stcont'] = pd.DataFrame.from_dict(Results[pf_str][cache_size]['nacho_pw_stcont'])
    PlotDataDF[ontime]['prowl'] = pd.DataFrame.from_dict(Results[pf_str][cache_size]['prowl'])
    PlotDataDF[ontime]['clank'] = pd.DataFrame.from_dict(Results[pf_str]['0-0']['clank'])

# Compute re-execution entry
for config in Configurations:
    for bench in Benchmarks:
        cont_cycles = PlotDataDF['0'][config][bench]['cycles']
        #print(cont_cycles)
        for ontime in OnTimes:
            #PlotDataDF[ontime][config][bench]['re_execution'] = PlotDataDF[ontime][config][bench]['cycles']/int(cont_cycles)
            df = PlotDataDF[ontime][config].T
            df['re_execution'] = df['cycles']/PlotDataDF['0'][config].T['cycles']
            # df['re_execution'] = df['restore_cycles'] + df['checkpoint_cycles']
            PlotDataDF[ontime][config] = df.T
            
# PlotDataDF['100000']['nacho_pw_stcont']
# print(PlotDataDF['250000']['nacho_pw_stcont'].T['cycles'])
# print(PlotDataDF['0']['nacho_pw_stcont'].T['cycles'])
# PlotDataDF['0']['nacho_pw_stcont'].T

In [8]:
# Create the latex table holding all the info

FREQ = 50000000 # 50MHz

Configurations = [
    'clank',
    'prowl',
    'nacho_pw_stcont'
]

Format = '{:.2f}'

def convertResult(val):
    return (val * 100)-100;

def convertOnTime(val):
    return 1/(FREQ/int(val))*1000

def toStr(val):
    s = Format.format(val)
    #if val < 10:
    #    s = r'\hphantom{0}' + s
    return s
    
def colorbox(exec_val, t):
    #return t
    perc = exec_val*5
    return r'\colorbox{hightlight!'+str(int(perc))+r'!white}{'+t+r'}'

tex=''
def tex_addl(t):
    global tex
    tex += t + '\n'
    
def tex_add(t):
    global tex
    tex += t
  
tex_addl(r'% table automatically generated by PowerFailures.ipynb')
tex_addl(r'% Assumed frequency: ' + str(FREQ/1000000) + ' MHz')

tex_addl(r'\begin{tabular}{@{}l|cccccc@{}}')
tex_addl(r'\toprule')

# Add the benchmark headers
tex_add(r'\textbf{On-duration}')
for bench in Benchmarks:
    tex_add(r' & \textbf{\textsf{'+BenchmarkNameMap[bench]+'}}')
tex_addl(r' \\ \midrule')

# Add the power-fail header
#tex_add(r'\textbf{On duration}')
#for bench in Benchmarks:
#    tex_add(r' & \textbf{\textsf{X}}')
#tex_addl(r' \\')

for ontime in OnTimes:
    if ontime == '0':
        continue
    
    ontime_str = '{:.0f}'.format(convertOnTime(ontime))+ '\,ms'
    tex_add(ontime_str)
    for bench in Benchmarks:
        re_val = convertResult(PlotDataDF[ontime]['nacho_pw_stcont'][bench]['re_execution'])
        re_val_str = toStr(re_val)
        tex_add(r' & '+colorbox(re_val,re_val_str+r'\%'))

    tex_addl(r' \\')

# Close the table
tex_addl(r' \bottomrule')
tex_addl(r'\end{tabular}')

print(tex)

# Save the table
with open('tables/benchmarks-reexecution-overhead.tex', "w") as f:
    # Writing data to a file
    f.write(tex)

% table automatically generated by PowerFailures.ipynb
% Assumed frequency: 50.0 MHz
\begin{tabular}{@{}l|cccccc@{}}
\toprule
\textbf{On-duration} & \textbf{\textsf{CoreMark}} & \textbf{\textsf{picojpeg}} & \textbf{\textsf{Tiny AES}} & \textbf{\textsf{CRC}} & \textbf{\textsf{Dijkstra}} & \textbf{\textsf{SHA}} & \textbf{\textsf{adpcm}} & \textbf{\textsf{quicksort}} & \textbf{\textsf{towers}} \\ \midrule
5\,ms & \colorbox{hightlight!31!white}{6.30\%} & \colorbox{hightlight!3!white}{0.65\%} & \colorbox{hightlight!12!white}{2.58\%} & \colorbox{hightlight!0!white}{0.00\%} & \colorbox{hightlight!3!white}{0.65\%} & \colorbox{hightlight!9!white}{1.98\%} & \colorbox{hightlight!32!white}{6.47\%} & \colorbox{hightlight!3!white}{0.80\%} & \colorbox{hightlight!1!white}{0.29\%} \\
10\,ms & \colorbox{hightlight!15!white}{3.02\%} & \colorbox{hightlight!1!white}{0.38\%} & \colorbox{hightlight!8!white}{1.78\%} & \colorbox{hightlight!0!white}{0.00\%} & \colorbox{hightlight!1!white}{0.30\%} & \colorbox{hi